# Partie 1 : Collecte et nettoyage des données

## Section 1 : Données financières

Notre but est simple : collecter les données financières des 4 dernières années pour toutes les entreprises du S&P500 et de l'EuroStoxx 600.

Pour parvenir à nos fins, nous devons procéder en deux temps :
- Collecter les 500 tickers (codes d'identification) des entreprises du S&P500 et les 600 tickers de l'EuroStoxx600.
- Utiliser les tickers pour collecter les données financières sur YahooFinance grâce à l'API non-officielle yfinance.

In [1]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

### Collecte des tickers S&P500

La liste des tickers se trouvant sur Wikipédia, un script de scrapping relativement simple nous permet de récupérer la liste des tickers des entreprises du S&P500.

In [2]:
sp500_tickers = []

try:
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    response.raise_for_status()

    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'id': 'constituents'})

    if table:
        for row in table.find_all('tr')[1:]: # On ignore l'en-tête
            cells = row.find_all('td')
            if cells:
                ticker = cells[0].text.strip()
                # Remplacement spécifique aux tickers américains pour yfinance (actions de classes spéciales)
                ticker = ticker.replace('.', '-')
                sp500_tickers.append(ticker)

    print(f"{len(sp500_tickers)} tickers S&P 500 ont été collectés.")
except Exception as e:
    print(f"Erreur lors de la récupération des tickers S&P 500: {e}")

503 tickers S&P 500 ont été collectés.


Observons rapidement les tickers collectés : 

In [3]:
print(sp500_tickers[:20])

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL']


Désormais, nous devons récupérer les tickers de l'EuroStoxx 600. Malheureusement, il n'existe pas de liste Wikipédia et nous sommes contraints d'utiliser une autre source : DividendMax. 

Nous faisons rapidement face à un problème. Les tickers YahooFinance ne sont pas standards : un suffixe correspondant à la place boursière européenne dans laquelle l'entreprise est cotée est ajouté par YahooFinance. 

Heureusement pour nous, DividendMax indique également la place de cotation. C'est pourquoi nous avons construit une rapide table de correspondance pour reconstruire les tickers scrappés au format YahooFinance.

In [4]:
exchange_to_suffix_map = {
    'EURONEXT AMSTERDAM': '.AS',
    'ATHENS EXCHANGE': '.AT',
    'EURONEXT BRUSSELS': '.BR',
    'COPENHAGEN': '.CO',
    'DUB LIN': '.IR',
    'FRANKFURT STOCK EXCHANGE': '.DE', 
    'HELSINKI STOCK EXCHANGE': '.HE',    
    'LISBON STOCK EXCHANGE': '.PT',      
    'LONDON STOCK EXCHANGE': '.L',       
    'MADRID STOCK EXCHANGE': '.MC',      
    'MILAN STOCK EXCHANGE': '.MI',       
    'OSLO STOCK EXCHANGE': '.OL',        
    'EURONEXT PARIS': '.PA',             
    'STOCKHOLM STOCK EXCHANGE': '.ST',   
    'SIX SWISS EXCHANGE': '.SW',         
    'VIENNA STOCK EXCHANGE': '.VI',      
    'XETRA': '.DE',                     
    'ITALIAN STOCK EXCHANGE': '.MI',     
    'IRISH STOCK EXCHANGE': '.IR',       
    'WARSAW STOCK EXCHANGE': '.WA',      
    'VALENCIA STOCK EXCHANGE': '.MC',    
    'NEW YORK STOCK EXCHANGE': '',       
    'BERLIN STOCK EXCHANGE': '.BE',      
    'LUXEMBOURG STOCK EXCHANGE': '.LU',  
    'JOHANNESBURG STOCK EXCHANGE': '.JO'
}

Pour mener à bien le scrapping, nous allons devoir naviguer sur une vingtaine de pages, récupérer les données cibles (ticker, nom, exchange) et les stocker dans un dictionnaire.

In [5]:
pip install lxml


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
from io import StringIO

stoxx600_companies_data = []
base_url = "https://www.dividendmax.com/market-index-constituents/stoxx600"
headers = {
    "User-Agent": "Mozilla/5.0 (compatible; ColabEuroStoxx600Scraper/1.0)"
}
page_num = 1
max_pages_to_check = 20 # on veut éviter les boucles infinies

while page_num <= max_pages_to_check:

    url = f"{base_url}?page={page_num}"
    
    try:
        response = requests.get(url, headers=headers, timeout=20)
        response.raise_for_status()

        tables = pd.read_html(StringIO(response.text))

        # On gère les cas limites, notamment la dernière page
        if not tables:
            print(f"  [Info] No tables found on page {page_num}. Assuming last page reached.")
            break

        df = tables[0]
        if df.empty:
                print(f"  [Info] Table on page {page_num} is empty. Assuming last page reached.")
                break

        df.columns = [c.strip().lower() for c in df.columns]

        ticker_col_name = "ticker"
        name_col_name = "company"
        exchange_col_name = "exchange"         

        page_companies_data = []
        
        for index, row in df.iterrows():
            
            ticker = str(row[ticker_col_name]).strip().upper()
            name = str(row[name_col_name]).strip()
            exchange = str(row[exchange_col_name]).strip().upper()

            if ticker and name and exchange: # On écarte les cellules vides
                # On nettoie le nom des tickers
                if ticker.endswith('.'):
                    cleaned_ticker = ticker[:-1]
                else:
                    cleaned_ticker = ticker

                # On récupère le suffixe pour construire le YahooTicker
                yahoo_suffix = exchange_to_suffix_map.get(exchange, '')
                yahoo_ticker = f"{cleaned_ticker}{yahoo_suffix}"


                # On construit le dictionnaire
                page_companies_data.append({
                    'Ticker': ticker,
                    'Nom': name,
                    'YahooTicker': yahoo_ticker
                })

        # On gère le cas de la dernière page
        if not page_companies_data:
                break

        stoxx600_companies_data.extend(page_companies_data)
        page_num += 1 # Move to the next page

    except requests.exceptions.RequestException as e:
        print(f"  [Erreur] Erreur lors de la récupération de la page {page_num}: {e}")
        break

    except Exception as e:
        print(f"  [Erreur] Erreur lors du parsing de la page {page_num}: {e}")
        break 


print(f"{len(stoxx600_companies_data)} tickers Eurostoxx 600 ont été collectés.")
print(stoxx600_companies_data)

565 tickers Eurostoxx 600 ont été collectés.
[{'Ticker': '1U1', 'Nom': '1&1 DrillischAktiengesellschaft', 'YahooTicker': '1U1.DE'}, {'Ticker': 'III', 'Nom': '3i Group plc', 'YahooTicker': 'III.L'}, {'Ticker': 'A2A', 'Nom': 'A2A Spa', 'YahooTicker': 'A2A.MI'}, {'Ticker': 'AAK', 'Nom': 'AAK AB', 'YahooTicker': 'AAK.ST'}, {'Ticker': 'AALB', 'Nom': 'Aalberts NV', 'YahooTicker': 'AALB.AS'}, {'Ticker': 'ARL', 'Nom': 'Aareal Beteiligungen AG', 'YahooTicker': 'ARL.DE'}, {'Ticker': 'ABBN', 'Nom': 'ABB Ltd.', 'YahooTicker': 'ABBN.SW'}, {'Ticker': 'ABDN', 'Nom': 'Aberdeen Group Plc', 'YahooTicker': 'ABDN.L'}, {'Ticker': 'ABN', 'Nom': 'ABN AMRO Bank N.V. - NLDR', 'YahooTicker': 'ABN.AS'}, {'Ticker': 'AC', 'Nom': 'Accor', 'YahooTicker': 'AC.PA'}, {'Ticker': 'ACKB', 'Nom': 'Ackermans & van Haaren NV', 'YahooTicker': 'ACKB.BR'}, {'Ticker': 'ACS', 'Nom': 'ACS, Actividades de Construccion Y Servicios, S.A.', 'YahooTicker': 'ACS.MC'}, {'Ticker': 'ADEN', 'Nom': 'Adecco Group AG', 'YahooTicker': 'ADEN.SW'

On a récupéré 565 tickers sur 600, dont toutes les plus grosses entreprises européennes. On s'en contentera : les quelques entreprises écartées sont plus petites et le sont à cause d'irrégularités dans le nom des tickers.

On peut maintenant passer à la deuxième étape : utiliser les tickers pour collecters les informations financières. On choisit ici de construire une fonction qui pourra être appelée pour construire les dataframe EuroStoxx et S&P500. L'objectif est de récupérer les données des trois états financiers (IS, CFS, BS) de chaque entreprise pour chaque année entre 2021 et 2024.

On a d'abord besoin de 3 fonctions préparatoires :

In [ ]:
def safe_div(a, b):
    """Division sécurisée, renvoie None si a ou b est None ou 0."""
    return a / b if a and b else None

def remove_tz(obj):
    """Supprime la timezone si tz-aware et index DatetimeIndex"""
    if isinstance(obj, pd.DataFrame) or isinstance(obj, pd.Series):
        if isinstance(obj.index, pd.DatetimeIndex) and obj.index.tz is not None:
            obj.index = obj.index.tz_localize(None)
    return obj

def parse_ticker(item, zone):
    """Retourne ticker_str, ticker_symbol, nom selon la zone."""
    if zone == 'USA':
        return item, item, item
    if zone == 'Eurostoxx' and isinstance(item, dict):
        ticker_str = item.get('Ticker', '')
        ticker_symbol = item.get('YahooTicker', '')
        nom = item.get('Nom', ticker_str)
        return ticker_str, ticker_symbol, nom
    return None, None, None

On peut désormais construire la fonction qui construit notre dataframe à partir de la liste des tickers :

In [8]:
def fetch_historical_fundamentals(ticker_list, zone='USA'):
    historical_data_list = []
    years_to_keep = range(2021, 2025)

    def remove_tz(obj):
        """Supprime la timezone si tz-aware"""
        if hasattr(obj, 'index') and obj.index.tz is not None:
            obj.index = obj.index.tz_localize(None)
        elif hasattr(obj, 'tz') and obj.tz is not None:
            obj = obj.tz_localize(None)
        return obj

    for i, item in enumerate(ticker_list):
        
        # --- 1. Détermination des tickers selon la zone ---
        ticker_str, ticker_symbol, nom = parse_ticker(item, zone)
 
        try:
            ticker = yf.Ticker(ticker_symbol)

            # --- 2. Données qualitatives ---
            info = getattr(ticker, 'info', {})
            sector_val = info.get('sector', 'N/A')
            industry_val = info.get('industry', 'N/A')
            country_val = info.get('country', 'N/A')
            beta_val = info.get('beta', None)

            # --- 3. Données financières ---
            financials_t = remove_tz(ticker.financials.transpose() if not ticker.financials.empty else pd.DataFrame())
            balance_sheet_t = remove_tz(ticker.balance_sheet.transpose() if not ticker.balance_sheet.empty else pd.DataFrame())
            dividends = remove_tz(getattr(ticker, 'dividends', pd.Series(dtype='float64')))

            # --- 4. Nombre d'actions en circulation ---
            try:
                shares_history = remove_tz(ticker.get_shares_full(start="2020-01-01"))
            except Exception:
                shares_history = pd.Series(dtype='float64')

            if financials_t.empty or balance_sheet_t.empty:
                continue

            # --- 5. Boucle temporelle sur les périodes financières ---
            for period_date in financials_t.index:
                if period_date.year not in years_to_keep:
                    continue
                if period_date not in balance_sheet_t.index:
                    continue

                fin_row = financials_t.loc[period_date]
                bs_row = balance_sheet_t.loc[period_date]

                # --- 6. Extraction des données financières ---
                net_income = fin_row.get('Net Income')
                total_revenue = fin_row.get('Total Revenue')
                basic_eps = fin_row.get('Basic EPS')

                total_equity = bs_row.get('Total Stockholder Equity') or bs_row.get('Stockholders Equity')
                total_assets = bs_row.get('Total Assets')
                total_debt = bs_row.get('Total Debt')

                # --- 7. Prix de l'action en fin d'année ---
                start_date = period_date.normalize()
                end_date = (period_date + pd.Timedelta(days=4)).normalize()
                price_data_eod = ticker.history(start=start_date, end=end_date)
                price_data_eod = remove_tz(price_data_eod)
                close_price = price_data_eod['Close'].iloc[0] if not price_data_eod.empty else None

                # --- 8. Market Cap historique ---
                shares_outstanding = shares_history.asof(period_date) if not shares_history.empty else None
                historical_market_cap = close_price * shares_outstanding if close_price and shares_outstanding else None

                # --- 9. Dividendes annuels ---
                year_start = pd.Timestamp(f'{period_date.year}-01-01')
                year_end = pd.Timestamp(f'{period_date.year}-12-31')
                yearly_dividends = dividends[(dividends.index >= year_start) & (dividends.index <= year_end)] if not dividends.empty else pd.Series(dtype='float64')
                annual_dividends = yearly_dividends.sum() if not yearly_dividends.empty else None

                # --- 10. Volume annuel échangé sur les marchés ---
                yearly_history = ticker.history(start=year_start, end=year_end)
                yearly_history = remove_tz(yearly_history)
                annual_volume_traded = yearly_history['Volume'].sum() if not yearly_history.empty and 'Volume' in yearly_history.columns else None

                # --- 11. Ratios financiers ---
                roe = safe_div(net_income, total_equity)
                roa = safe_div(net_income, total_assets)
                debt_to_equity = safe_div(total_debt, historical_market_cap)
                per_calcule = safe_div(close_price, basic_eps)

                # --- 12. Construction du dictionnaire final ---
                historical_data_list.append({
                    'Ticker': ticker_str,
                    'YahooTicker': ticker_symbol,
                    'Nom': nom,
                    'Zone': zone,
                    'Sector': sector_val,
                    'Industry': industry_val,
                    'Country': country_val,
                    'Beta': beta_val,
                    'MarketCapitalization': historical_market_cap,
                    'SharesOutstanding': shares_outstanding,
                    'AnneeFiscale': period_date.year,
                    'DateRapport': period_date.strftime('%Y-%m-%d'),
                    'NetIncome': net_income,
                    'TotalRevenue': total_revenue,
                    'TotalEquity': total_equity,
                    'TotalAssets': total_assets,
                    'TotalDebt': total_debt,
                    'EPS': basic_eps,
                    'EOD_Close_Price': close_price,
                    'ROE_calcule': roe,
                    'ROA_calcule': roa,
                    'DebtToEquity_calcule': debt_to_equity,
                    'PER_calcule': per_calcule,
                    'Dividendes_Annuels': annual_dividends,
                    'Annual_Volume_Traded': annual_volume_traded
                })

        except Exception as e_outer:
            print(f"[Grave Erreur] {ticker_symbol}: {e_outer}")

    if not historical_data_list:
        return None

    # --- 13. Création et tri du DataFrame final ---
    df_historical = pd.DataFrame(historical_data_list)
    df_historical = df_historical.sort_values(by=['Ticker', 'AnneeFiscale'], ascending=[True, False])
    return df_historical

On peut maintenant construire notre dataframe pour l'EuroStoxx600 :

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

df_eurostoxx600 = fetch_historical_fundamentals(stoxx600_companies_data, zone='Eurostoxx')

df_eurostoxx600.to_csv('df_eurostoxx600_raw.csv', index=False, encoding='utf-8-sig')

On vérifie rapidement la cohérence du résultat :

In [16]:
display(df_eurostoxx600.sample(10))

,Ticker,YahooTicker,Nom,Zone,Sector,Industry,Country,Beta,MarketCapitalization,SharesOutstanding,AnneeFiscale,DateRapport,NetIncome,TotalRevenue,TotalEquity,TotalAssets,TotalDebt,EPS,EOD_Close_Price,ROE_calcule,ROA_calcule,DebtToEquity_calcule,PER_calcule,Dividendes_Annuels,Annual_Volume_Traded
1224,PKN,PKN.WA,Orlen S.A.,Eurostoxx,Energy,Oil & Gas Integrated,Poland,0.749,6.648492e+10,1.171750e+09,2023,2023-12-31,2.092200e+10,3.719160e+11,1.523220e+11,2.644630e+11,2.589600e+10,18.0200,56.739849,0.137354,0.079111,0.389502,3.148715,5.500000,551515817
919,ITRK,ITRK.L,Intertek Group plc,Eurostoxx,Industrials,Specialty Business Services,United Kingdom,0.678,6.639467e+11,1.624330e+08,2022,2022-12-31,2.888000e+08,3.192900e+09,1.284000e+09,3.660300e+09,1.380800e+09,1.7920,4087.511230,0.224922,0.078901,0.002080,2280.977249,105.800000,100616231
748,GLEN,GLEN.L,Glencore plc,Eurostoxx,Basic Materials,Other Industrial Metals & Mining,Switzerland,0.587,5.722696e+12,1.221600e+10,2023,2023-12-31,4.280000e+09,2.178290e+11,4.358000e+10,1.238690e+11,3.224100e+10,0.3400,468.459076,0.098210,0.034553,0.005634,1377.820812,41.684899,9828693201
260,BAS,BAS.DE,Basf SE,Eurostoxx,Basic Materials,Chemicals,Germany,0.923,3.880387e+10,8.925220e+08,2023,2023-12-31,2.250000e+08,6.890200e+10,3.527700e+10,7.739500e+10,2.134700e+10,0.2500,43.476658,0.006378,0.002907,0.550125,173.906631,3.400000,640549610
519,DTE,DTE.DE,Deutsche Telekom AG,Eurostoxx,Communication Services,Telecom Services,Germany,0.283,1.410901e+11,4.978620e+09,2024,2024-12-31,1.120900e+10,1.151110e+11,6.329600e+10,3.049340e+11,1.471160e+11,2.2700,28.339209,0.177089,0.036759,1.042709,12.484233,0.770000,1652857799
646,FERG,FERG,Ferguson Plc.,Eurostoxx,Industrials,Industrial Distribution,United States,1.129,2.856604e+10,2.236390e+08,2021,2021-07-31,NaN,NaN,NaN,NaN,NaN,NaN,127.732826,NaN,NaN,NaN,NaN,4.194000,4864900
570,ENG,ENG.MC,Enagas SA,Eurostoxx,Utilities,Utilities - Regulated Gas,Spain,0.352,3.808176e+09,2.614880e+08,2021,2021-12-31,4.038260e+08,9.756860e+08,3.085430e+09,9.873718e+09,5.259712e+09,1.5443,14.563484,0.130882,0.040899,1.381163,9.430476,1.688000,268580473
278,BEZ,BEZ.L,Beazley Plc.,Eurostoxx,Financial Services,Insurance - Specialty,United Kingdom,0.136,2.838174e+11,6.092410e+08,2021,2021-12-31,3.087000e+08,3.284700e+09,2.190200e+09,1.005390e+10,8.430000e+07,0.5090,465.854095,0.140946,0.030705,0.000297,915.233979,NaN,425545104
997,KBX,KBX.DE,Knorr-Bremse AG,Eurostoxx,Consumer Cyclical,Auto Parts,Germany,1.083,9.097104e+09,1.615840e+08,2023,2023-12-31,5.530000e+08,7.926000e+09,2.836000e+09,8.249000e+09,NaN,3.4300,56.299538,0.194993,0.067038,NaN,16.413859,1.450000,43260111
862,IGG,IGG.L,IG Group Holdings Plc,Eurostoxx,Financial Services,Capital Markets,United Kingdom,0.698,3.019715e+11,3.730940e+08,2024,2024-05-31,3.077000e+08,8.524000e+08,1.889500e+09,2.940900e+09,3.232000e+08,0.7940,809.371033,0.162847,0.104628,0.001070,1019.358983,46.200000,306790601


On fait la même chose pour le S&P500 :

In [ ]:
pd.set_option('display.max_rows', None)   
pd.set_option('display.max_columns', None)

df_sp500 = fetch_historical_fundamentals(sp500_tickers, zone='USA')

df_sp500.to_csv('df_sp500_raw.csv', index=False, encoding='utf-8-sig')

$CEG: possibly delisted; no price data found  (1d 2021-12-31 00:00:00 -> 2022-01-04 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1640926800, endDate = 1641272400")


[Grave Erreur] CEG: 'Index' object has no attribute 'tz'


$GEHC: possibly delisted; no price data found  (1d 2021-12-31 00:00:00 -> 2022-01-04 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1640926800, endDate = 1641272400")


[Grave Erreur] GEHC: 'Index' object has no attribute 'tz'


$GEV: possibly delisted; no price data found  (1d 2023-12-31 00:00:00 -> 2024-01-04 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1703998800, endDate = 1704344400")


...

[Grave Erreur] SOLS: 'Index' object has no attribute 'tz'


$SOLV: possibly delisted; no price data found  (1d 2023-12-31 00:00:00 -> 2024-01-04 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1703998800, endDate = 1704344400")


[Grave Erreur] SOLV: 'Index' object has no attribute 'tz'


$VLTO: possibly delisted; no price data found  (1d 2022-12-31 00:00:00 -> 2023-01-04 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1672462800, endDate = 1672808400")


[Grave Erreur] VLTO: 'Index' object has no attribute 'tz'


On vérifie rapidement la cohérence du résultat :

In [20]:
display(df_sp500.sample(10))

,Ticker,YahooTicker,Nom,Zone,Sector,Industry,Country,Beta,MarketCapitalization,SharesOutstanding,AnneeFiscale,DateRapport,NetIncome,TotalRevenue,TotalEquity,TotalAssets,TotalDebt,EPS,EOD_Close_Price,ROE_calcule,ROA_calcule,DebtToEquity_calcule,PER_calcule,Dividendes_Annuels,Annual_Volume_Traded
712,EXC,EXC,EXC,USA,Utilities,Utilities - Regulated Electric,United States,0.550,3.834748e+10,9.937420e+08,2022,2022-12-31,2.170000e+09,1.907800e+10,2.474400e+10,9.534900e+10,4.005000e+10,2.183099,38.588974,0.087698,0.022758,1.044397,17.676236,1.352,2060996695
1517,RTX,RTX,RTX,USA,Industrials,Aerospace & Defense,United States,0.435,1.385105e+11,1.470060e+09,2022,2022-12-31,5.197000e+09,6.707400e+10,7.263200e+10,1.588640e+11,3.350000e+10,3.520000,94.221016,0.071552,0.032714,0.241859,26.767334,2.160,1318403000
1696,TDY,TDY,TDY,USA,Technology,Scientific & Technical Instruments,United States,1.031,2.068105e+10,4.718470e+07,2023,2023-12-31,8.857000e+08,5.635500e+09,9.221200e+09,1.452790e+10,3.244900e+09,18.712560,438.299988,0.096050,0.060965,0.156902,23.422770,NaN,60398300
7,AOS,AOS,AOS,USA,Industrials,Specialty Industrial Machinery,United States,1.335,1.267607e+10,1.588730e+08,2021,2021-12-31,4.871000e+08,3.538900e+09,1.832200e+09,3.474400e+09,2.190000e+08,3.089705,79.787468,0.265855,0.140197,0.017277,25.823652,1.060,263874400
711,EXC,EXC,EXC,USA,Utilities,Utilities - Regulated Electric,United States,0.550,3.362835e+10,9.954370e+08,2023,2023-12-31,2.328000e+09,2.172700e+10,2.575500e+10,1.018560e+11,4.400800e+10,2.330330,33.782497,0.090390,0.022856,1.308658,14.496873,1.440,1661434700
1506,DGX,DGX,DGX,USA,Healthcare,Diagnostics & Research,United States,0.616,1.670082e+10,1.138870e+08,2022,2022-12-31,9.460000e+08,9.883000e+09,5.893000e+09,1.283700e+10,4.714000e+09,8.100000,146.643768,0.160529,0.073693,0.282262,18.104169,2.600,269568900
1332,NRG,NRG,NRG,USA,Utilities,Utilities - Independent Power Producers,United States,1.253,1.115453e+10,2.257640e+08,2023,2023-12-31,-2.020000e+08,2.882300e+10,2.906000e+09,2.603800e+10,1.097100e+10,-1.120000,49.407921,-0.069511,-0.007758,0.983547,-44.114215,1.512,1004167400
1523,REG,REG,REG,USA,Real Estate,REIT - Retail,United States,0.940,1.288746e+10,1.815050e+08,2024,2024-12-31,4.003880e+08,1.453904e+09,6.724146e+09,1.239196e+10,5.018169e+09,2.138066,71.003357,0.059545,0.032310,0.389384,33.209151,2.715,277718500
1501,QCOM,QCOM,QCOM,USA,Technology,Semiconductors,United States,1.214,1.181761e+11,1.116000e+09,2023,2023-09-30,7.232000e+09,3.582000e+10,2.158100e+10,5.104000e+10,1.539800e+10,6.470000,105.892540,0.335110,0.141693,0.130297,16.366699,3.150,2028691900
159,AMAT,AMAT,AMAT,USA,Technology,Semiconductor Equipment & Materials,United States,1.666,1.212941e+11,9.029290e+08,2021,2021-10-31,NaN,NaN,NaN,NaN,NaN,NaN,134.333984,NaN,NaN,NaN,NaN,0.940,2047499900


### Passons au nettoyage de ces deux dataframes

## Section 2 : Données Damodaran

Après réflexion et premières tentatives de modélisation en utilisant uniquement les données financières des entreprises, nous avons décidé d'enrichir notre analyse en utilisant des données macro-économiques : *default spread, equity risk premium, country risk premium, corporate task et Moody's rating*. 

Ces données se trouvent sur le site du professeur Damodaran, que nous allons scrapper.

(Source : https://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/ctryprem.html)

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
url = "https://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/ctryprem.html"

On effectue notre tentative de scrapping. L'objectif est de reproduire le tableau disponible sur le site :

In [ ]:
try:
    # On télécharge les informations du site
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    response.raise_for_status()

    # On identifie la table
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'width': '919'})

    # On récupère toutes les lignes de la table
    trs = table.find_all('tr')

    # On trouve la ligne correspondant au 'Country'
    header_row = None
    for tr in trs:
        tds = tr.find_all('td')
        if tds and 'Country' in tds[0].get_text():
            header_row = tr
            break

    # On extrait les headers
    headers = [td.get_text(strip=True) for td in header_row.find_all('td')]

    # On garde uniquement les lignes avec des données :
    rows = []
    for tr in table.find_all('tr'):
        tds = tr.find_all('td')
        if len(tds) == 6 and tr != header_row:
            row = [td.get_text(strip=True).replace('\xa0','').replace('\n',' ').strip() for td in tds]
            rows.append(row)

except Exception as e:
    print("Erreur lors du scraping :", e)

# On créé notre dataframe
df_damodaran = pd.DataFrame(rows, columns=headers)


On observe un échantillon du dataframe obtenu :

In [ ]:
print (df_damodaran.head())

                                   Country Adj. Default\n              Spread  \
0                                Abu Dhabi                              0.49%   
1                                  Albania                              3.56%   
2                                  Algeria                              2.98%   
3  Andorra               (Principality of)                              1.58%   
4                                   Angola                              6.44%   

  Equity Risk\n              Premium Country Risk\n              Premium  \
0                              4.99%                               0.66%   
1                              9.13%                               4.80%   
2                              8.35%                               4.02%   
3                              6.46%                               2.13%   
4                             13.01%                               8.68%   

  Corporate Tax\n              Rate Moody's rating  
0  

On constate qu'il y a du ménage à faire : il faut nettoyer les noms de colonnes, définir les colonnes numériques et les convertir, définir les colonnes de texte et supprimer les lignes vides.

In [ ]:
# Nettoyage des noms de colonnes
df_damodaran.columns = (
    df_damodaran.columns
    .str.replace('\n', ' ', regex=False)
    .str.replace(r'\s+', ' ', regex=True)
    .str.strip()
)

# Définition des colonnes numériques
numeric_cols = [
    'Adj. Default Spread',
    'Equity Risk Premium',
    'Country Risk Premium',
    'Corporate Tax Rate',
]

# Conversion de ces colonnes en numérique
for col in numeric_cols:
    df_damodaran[col] = (
        df_damodaran[col]
        .astype(str)
        .str.replace('%', '', regex=False)
        .str.replace(',', '', regex=False)
        .str.replace(r'\s+', ' ', regex=True)
        .str.strip()
    )
    df_damodaran[col] = pd.to_numeric(df_damodaran[col], errors='coerce')

# Définition des colonnes de texte
df_damodaran['Country'] = df_damodaran['Country'].astype(str).str.strip()

# Suppression des lignes vides
df_damodaran = df_damodaran.dropna(how='all')

On vérifie le résultat obtenu :

In [ ]:
print(df_damodaran.head())
print(df_damodaran.columns.tolist())

                                   Country  Adj. Default Spread  \
0                                Abu Dhabi                 0.49   
1                                  Albania                 3.56   
2                                  Algeria                 2.98   
3  Andorra               (Principality of)                 1.58   
4                                   Angola                 6.44   

   Equity Risk Premium  Country Risk Premium  Corporate Tax Rate  \
0                 4.99                  0.66               15.00   
1                 9.13                  4.80               15.00   
2                 8.35                  4.02               15.00   
3                 6.46                  2.13               18.98   
4                13.01                  8.68               25.00   

  Moody's rating  
0            Aa2  
1            Ba3  
2             NR  
3           Baa1  
4             B3  
['Country', 'Adj. Default Spread', 'Equity Risk Premium', 'Country Risk Pr

Par mesure de précaution, on sauvegarde ces données dans notre dossier data :

In [ ]:
df_damodaran.to_csv("data/country_risk_premiums.csv", index=False)

Le dataframe s'affiche parfaitement :

In [ ]:
df_test = pd.read_csv("data/country_risk_premiums.csv")
df_test.head(20)   

,Country,Adj. Default Spread,Equity Risk Premium,Country Risk Premium,Corporate Tax Rate,Moody's rating
0,Abu Dhabi,0.49,4.99,0.66,15.00,Aa2
1,Albania,3.56,9.13,4.80,15.00,Ba3
2,Algeria,2.98,8.35,4.02,15.00,NR
3,Andorra (Principality of),1.58,6.46,2.13,18.98,Baa1
4,Angola,6.44,13.01,8.68,25.00,B3
5,Anguilla,6.01,12.44,8.11,25.23,NR
6,Antigua & Barbuda,6.01,12.43,8.10,25.23,NR
7,Argentina,11.88,20.35,16.02,35.00,Ca
8,Armenia,3.56,9.13,4.80,18.00,Ba3
9,Aruba,2.18,7.26,2.93,25.00,Baa3
